In [90]:
import pygame
from cellitaire.game.game import Game
from cellitaire.game.slot import Slot
from cellitaire.game.card import Card

In [141]:
SCREEN_WIDTH = 1920
SCREEN_HEIGHT = 1080
SCREEN_DIMS = (SCREEN_WIDTH, SCREEN_HEIGHT)

RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
WHITE = (255, 255, 255)
PURPLE = (112, 1, 169)
YELLOW = (255, 255, 0)
BLACK = (0, 0, 0)

BOARD_MARGIN = 15

SLOT_WIDTH = 120
SLOT_HEIGHT = 120
SLOT_PADDING = 5
SLOT_BACKGROUND_COLOR = GREEN
SLOT_LONELY_OR_SUFFOCATED_COLOR = PURPLE
SLOT_PLACEABLE_COLOR = YELLOW
SLOT_HOVER_COLOR = (92, 152, 4)

BACKGROUND_COLOR = BLUE

GU_SLOT_UPDATE = pygame.USEREVENT + 1
GU_FOUNDATION_UPDATE = pygame.USEREVENT + 2
GU_STOCKPILE_UPDATE = pygame.USEREVENT + 3

SLOT_CLICKED = pygame.USEREVENT + 4

In [142]:
class GameWrapper:
    def __init__(self, rows, cols, num_reserved):
        self.rows = rows
        self.cols = cols
        self.num_reserved = num_reserved
        
        self.game = None

    def reset(self):
        self.game = Game()
        self.game.new_game(self.rows, self.cols, self.num_reserved)

    def publish_updates(self):
        slot_events = [
            pygame.event.Event(
                GU_SLOT_UPDATE,
                coordinate=(i, j),
                card=slot.card, 
                is_lonenly=slot.is_lonely, 
                is_suffocated=slot.is_suffocated,
                is_placeable=slot.is_placeable
            ) for i, row in enumerate(self.game.board.slots) for j, slot in enumerate(row)
        ]

        for event in slot_events:
            pygame.event.post(event)

        pygame.event.post(pygame.event.Event(GU_STOCKPILE_UPDATE, top_card=self.game.stockpile.top_card()))
        
        pygame.event.post(pygame.event.Event(GU_FOUNDATION_UPDATE, foundation_dict=self.game.foundation.foundation))

    def make_move(self, move):
        self.game.make_move(move)
        self.publish_updates()
        
        

In [143]:
class SlotSprite(pygame.sprite.Sprite):
    def __init__(self, height, width, x, y, coordinate):
        super().__init__()
        
        self.image = pygame.Surface([width, height])
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y

        self.height = height
        self.width = width

        self.coordinate = coordinate

        self.card = None
        self.is_lonely = False
        self.is_suffocated = False
        self.is_placeable = False
        self.is_hovered = False

        self.draw_slot()

    def draw_background(self):
        pygame.draw.rect(self.image, SLOT_BACKGROUND_COLOR, pygame.Rect(0, 0, self.width, self.height))

    def draw_outline(self):
        if not (self.is_lonely or self.is_suffocated or self.is_placeable):
            return

        outline_color = SLOT_LONELY_OR_SUFFOCATED_COLOR
        if self.is_placeable:
            outline_color = SLOT_PLACEABLE_COLOR

        pygame.draw.rect(
            self.image, 
            outline_color, 
            pygame.Rect(
                0, 
                0, 
                self.width, 
                self.height
            ),
            SLOT_PADDING
        )

    def draw_card(self):
        if self.card is None:
            return
        card_rect = pygame.draw.rect(
            self.image,
            WHITE,
            pygame.Rect(
                SLOT_PADDING, 
                SLOT_PADDING, 
                self.width - 2 * SLOT_PADDING, 
                self.height - 2 * SLOT_PADDING
            )
        )

        font = pygame.font.Font(None, 24)
        card_text = str(self.card)
        text_surface = font.render(card_text, True, BLACK)
        text_rect = text_surface.get_rect(center=card_rect.center)
        self.image.blit(text_surface, text_rect)

    def draw_hover_overlay(self):
        if not self.is_hovered:
            return
        overlay = pygame.Surface((self.width, self.height), pygame.SRCALPHA)
        overlay.fill((0, 0, 0, 50))
        self.image.blit(overlay, (0, 0))
        self.rect = self.image.get_rect(topleft=self.rect.topleft)

    def draw_slot(self):
        self.draw_background()
        self.draw_card()
        self.draw_outline()
        self.draw_hover_overlay()

    def handle_slot_update_event(self, event):
        self.card = event.card
        self.is_lonely = event.is_lonenly
        self.is_suffocated = event.is_suffocated
        self.is_placeable = event.is_placeable

    def handle_clicked(self):
        if not (self.is_lonely or self.is_suffocated or self.is_placeable):
            return
        pygame.event.post(pygame.event.Event(SLOT_CLICKED, coordinate=self.coordinate))
    
    def update(self, events):
        mouse_pos = pygame.mouse.get_pos()
        if self.rect.collidepoint(mouse_pos):
            self.is_hovered = True
        else:
            self.is_hovered = False

        for event in events:
            if event.type == GU_SLOT_UPDATE and  event.coordinate == self.coordinate:
                self.handle_slot_update_event(event)
            if self.is_hovered and event.type == pygame.MOUSEBUTTONUP:
                self.handle_clicked()

        self.draw_slot()

In [ ]:
screen = pygame.display.set_mode(SCREEN_DIMS)
pygame.display.set_caption("Cellitaire RL")

pygame.init()

gw = GameWrapper(7, 12, 6)
gw.reset()

testCard = Card(1)

all_sprites = pygame.sprite.Group()

for i in range(7):
    for j in range(12):
        all_sprites.add(
            SlotSprite(
                height=SLOT_HEIGHT,
                width=SLOT_WIDTH, 
                x=BOARD_MARGIN + j * SLOT_WIDTH, 
                y=BOARD_MARGIN + i * SLOT_HEIGHT,
                coordinate=(i, j)
            )
        )

running = True
clock = pygame.time.Clock() 
    
while running:
    events = pygame.event.get()
    for event in events:
        if event.type == pygame.QUIT:
            running = False
        if event.type == SLOT_CLICKED:
            gw.make_move(event.coordinate)

    gw.publish_updates()
    
    all_sprites.update(events)
    screen.fill(BACKGROUND_COLOR)
    all_sprites.draw(screen)
    pygame.display.flip()
    clock.tick(30)

pygame.quit()

In [133]:
pygame.quit()